In [ ]:
from scipy.spatial import distance #computes distance
from imutils import face_utils #imutils is used for basic image processing
import numpy as np #used for working with arrays
import pygame #For playing sound
import time # used for time related functions
import dlib # used for face detection
import cv2 as cv # used to load image 

In [ ]:
#Initialize Pygame and load music
pygame.mixer.init()
pygame.mixer.music.load('C:\\Users\\DELL\\Desktop\\Driver-Drowsiness-Detector-master\\audio\\alert.wav')

In [ ]:
#Minimum threshold of eye aspect ratio below which alarm is triggerd
EYE_ASPECT_RATIO_THRESHOLD = 0.3

In [ ]:
#Minimum consecutive frames for which eye ratio is below threshold for alarm to be triggered
EYE_ASPECT_RATIO_CONSEC_FRAMES = 50

In [ ]:
#COunts no. of consecutuve frames below threshold value
COUNTER = 0

In [ ]:
#Load face cascade which will be used to draw a rectangle around detected faces.
face_cascade = cv.CascadeClassifier("C:\\Users\\DELL\\Desktop\\Driver-Drowsiness-Detector-master\\haarcascades\\haarcascade_frontalface_default.xml")


In [ ]:
#This function calculates and return eye aspect ratio
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])

    ear = (A+B) / (2*C)
    return ear

In [ ]:
#Load face detector and predictor, uses dlib shape predictor file
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('C:\\Users\\DELL\\Desktop\\Driver-Drowsiness-Detector-master\\shape_predictor_68_face_landmarks.dat')

In [ ]:
#Extract indexes of facial landmarks for the left and right eye
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS['left_eye']
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS['right_eye']



In [ ]:
#Start webcam video capture
video_capture = cv.VideoCapture(0)

In [ ]:
#Give some time for camera to initialize(not required)
time.sleep(2)

In [ ]:
while(True):
    #Read each frame and flip it, and convert to grayscale
    ret, frame = video_capture.read()
    frame = cv.flip(frame,1)
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    #Detect facial points through detector function
    faces = detector(gray, 0)

    #Detect faces through haarcascade_frontalface_default.xml
    face_rectangle = face_cascade.detectMultiScale(gray, 1.3, 5)

    #Draw rectangle around each face detected
    for (x,y,w,h) in face_rectangle:
        cv.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)

    #Detect facial points
    for face in faces:

        shape = predictor(gray, face)
        shape = face_utils.shape_to_np(shape)

        #Get array of coordinates of leftEye and rightEye
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]

        #Calculate aspect ratio of both eyes
        leftEyeAspectRatio = eye_aspect_ratio(leftEye)
        rightEyeAspectRatio = eye_aspect_ratio(rightEye)

        eyeAspectRatio = (leftEyeAspectRatio + rightEyeAspectRatio) / 2

        #Use hull to remove convex contour discrepencies and draw eye shape around eyes
        leftEyeHull = cv.convexHull(leftEye)
        rightEyeHull = cv.convexHull(rightEye)
        cv.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

        #Detect if eye aspect ratio is less than threshold
        if(eyeAspectRatio < EYE_ASPECT_RATIO_THRESHOLD):
            COUNTER += 1
            #If no. of frames is greater than threshold frames,
            if COUNTER >= EYE_ASPECT_RATIO_CONSEC_FRAMES:
                pygame.mixer.music.play(-1)
                cv.putText(frame, "You are Drowsy", (150,200), cv.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,255), 2)
        else:
            pygame.mixer.music.stop()
            COUNTER = 0

    #Show video feed
    cv.imshow('Video', frame)
    if(cv.waitKey(1) & 0xFF == ord('q')):
        break

In [ ]:
#Finally when video capture is over, release the video capture and destroyAllWindows
video_capture.release()
cv.destroyAllWindows()